# Web Scraping

There is an ethical way to scrape websites. Simply check out the `/robots.txt` file and/or check to see if there is an API.

The standard library for web scraping is [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).

In [1]:
import requests
from bs4 import BeautifulSoup

In [4]:
res = requests.get('https://news.ycombinator.com/news')
soup = BeautifulSoup(res.text, 'html.parser')